In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from func import MlChart, PipeModel
import numpy as np

In [25]:
dataset = pd.read_csv('Walmart_Sales.csv')
dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [3]:
date = dataset['Date']
date.head()

0    05-02-2010
1    12-02-2010
2    19-02-2010
3    26-02-2010
4    05-03-2010
Name: Date, dtype: object

In [4]:
df = dataset.drop(dataset[['Store', 'Date']], axis = 1) # , 'Holiday_Flag', 'Fuel_Price', 'Temperature', 'CPI'
df.head()

,Weekly_Sales,Unemployment
0,1643690.90,8.106
1,1641957.44,8.106
2,1611968.17,8.106
3,1409727.59,8.106
4,1554806.68,8.106


In [5]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]
print(f'Dependent Variable \n{X.head()}\n')
print(f'Target \n{y.head()}\n')

Dependent Variable 
   Unemployment
0         8.106
1         8.106
2         8.106
3         8.106
4         8.106

Target 
0    1643690.90
1    1641957.44
2    1611968.17
3    1409727.59
4    1554806.68
Name: Weekly_Sales, dtype: float64



## Forecasting

In [30]:
dataset['Date'] = pd.to_datetime(dataset['Date'],format = '%d-%m-%Y')
dataset.index = dataset['Date']
invalid_dates = dataset[(dataset['Date'] < '0001-01-01') | (dataset['Date'] > '9999-12-31')]
print(invalid_dates)
dataset.info()

Empty DataFrame
Columns: [Store, Date, Weekly_Sales, Holiday_Flag, Temperature, Fuel_Price, CPI, Unemployment]
Index: []
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6435 entries, 2010-02-05 to 2012-10-26
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         6435 non-null   int64         
 1   Date          6435 non-null   datetime64[ns]
 2   Weekly_Sales  6435 non-null   float64       
 3   Holiday_Flag  6435 non-null   int64         
 4   Temperature   6435 non-null   float64       
 5   Fuel_Price    6435 non-null   float64       
 6   CPI           6435 non-null   float64       
 7   Unemployment  6435 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 452.5 KB


In [31]:
import seaborn as sns
color_pal = sns.color_palette()
dataset.plot(style = '.', figsize=(16,6),
             color = color_pal[0])
plt.show()

ValueError: Date ordinal 4009620.8755 converts to 12947-12-20T21:00:43.200000 (using epoch 1970-01-01T00:00:00), but Matplotlib dates must be between year 0001 and 9999.

<Figure size 1600x600 with 1 Axes>

# Machine Learning

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import (
        RandomForestRegressor, 
        AdaBoostRegressor, 
        GradientBoostingRegressor,
        HistGradientBoostingRegressor,
        StackingRegressor
        )
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import GridSearchCV

## Split The Data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15,
                                                    random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2,
                                                  random_state=42)

# Print the shape
print(f'Training batch shape X shape: {X_train.shape} target shape: {y_train.shape}')
print(f'Test batch shape X shape: {X_test.shape} target shape: {y_test.shape}')
print(f'Validation batch shape X shape: {X_val.shape} target shape: {y_val.shape}')

Training batch shape X shape: (4375, 1) target shape: (4375,)
Test batch shape X shape: (966, 1) target shape: (966,)
Validation batch shape X shape: (1094, 1) target shape: (1094,)


In [10]:
# Convert to numpy.ndarray and Reshape the target
X_train = X_train.values
X_val = X_val.values
X_test = X_test.values
y_train = y_train.values.reshape(-1, 1)
y_val = y_val.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Transform the target
X_train_log = np.log1p(X_train)
X_val_log = np.log1p(X_val)
X_test_log = np.log1p(X_test)
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)
y_test_log = np.log1p(y_test)

# SQRT
X_train_sqrt = np.sqrt(X_train)
X_val_sqrt = np.sqrt(X_val)
X_test_sqrt = np.sqrt(X_test)
y_train_sqrt = np.sqrt(y_train)
y_val_sqrt = np.sqrt(y_val)
y_test_sqrt = np.sqrt(y_test)

In [11]:
import warnings
warnings.filterwarnings('ignore')
models = [RandomForestRegressor(n_estimators=100, 
                                random_state=42),
          AdaBoostRegressor(estimator=DecisionTreeRegressor(), n_estimators=50)]#,
          # GradientBoostingRegressor(n_estimators=100),
          # HistGradientBoostingRegressor(max_iter=100),
          # StackingRegressor(estimators=[('rf', RandomForestRegressor()), ('gb', GradientBoostingRegressor())], final_estimator=RidgeCV())]

print('Normal Transform:')
RunModels = PipeModel(models, 
                      X_train, y_train,
                      X_val, y_val)

print('\nLog Transform:')
RunModels_log = PipeModel(models, 
                      X_train, y_train_log,
                      X_val, y_val_log)

print('\nSQRT Transform:')
RunModels_sqrt = PipeModel(models, 
                      X_train_sqrt, y_train_sqrt,
                      X_val_sqrt, y_val_sqrt)




Normal Transform:
RandomForestRegressor score: 45.6165%
AdaBoostRegressor score: 41.7866%

Log Transform:
RandomForestRegressor score: 52.1937%
AdaBoostRegressor score: 50.9977%

SQRT Transform:
RandomForestRegressor score: 48.2821%
AdaBoostRegressor score: 46.8596%


In [14]:
# Tuning
import warnings
warnings.filterwarnings('ignore')
Params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

rf = RandomForestRegressor()
y_train_log = y_train_log.ravel()
# Grid search
grid_search = GridSearchCV(estimator=rf, param_grid=Params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train_log)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=

# Visualization

In [ ]:
# Declare the models
for model in models:
    pipe = make_pipeline(StandardScaler(), model)
    pipe.fit(X_train, y_train)
    MlChart(model = pipe,
            X_train = X_train, 
            y_train = y_train_log,
            title = model,
            target = y_val_log, 
            prediction = X_val,
            ResidualPlot=True)

In [ ]:
# Declare the models
for model in models:
    pipe = make_pipeline(StandardScaler(), model)
    pipe.fit(X_train, y_train)
    MlChart(model = pipe,
            X_train = X_train, 
            y_train = y_train,
            title = model,
            target = y_val, 
            prediction = X_val,
            ComparisonPlot=True,
            LearningCurve=True)